# Task 1: Fine-tune Chemical Language Model

The goal is to fine-tune a pre-trained chemical language model on a regression task using the Lipophilicity dataset. The task involves predicting the lipophilicity value for a given molecule representation (SMILES string). You will learn how to load and tokenize a dataset from HuggingFace, how to load a pre-trained language model, and finally, how to run a model in inference mode.

Your task is to complete the missing code blocks below.

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00


In [3]:
!pip install --upgrade --force-reinstall transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.0/763.0 kB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import transformers
print(transformers.__version__)

4.49.0


In [2]:
# import dependencies
import torch
from datasets import load_dataset
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm.notebook import tqdm
import random

# 1.Fine-tune a Chemical Language Model on Lipophilicity


## --- Step 1: Load Dataset ---

The dataset we are going to use is the [Lipophilicity](https://huggingface.co/datasets/scikit-fingerprints/MoleculeNet_Lipophilicity) dataset, part of [MoleculeNet](https://pubs.rsc.org/en/content/articlelanding/2018/sc/c7sc02664a) benchmark.

Lipophilicity, also known as hydrophobicity, is a measure of how readily a substance dissolves in nonpolar solvents (such as oil) compared to polar solvents (such as water).

In [3]:
# specify dataset name and model name
DATASET_PATH = "scikit-fingerprints/MoleculeNet_Lipophilicity"
MODEL_NAME = "ibm/MoLFormer-XL-both-10pct"  #MoLFormer model

In [4]:
# load the dataset from HuggingFace
dataset = load_dataset(DATASET_PATH)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

lipophilicity.csv:   0%|          | 0.00/223k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4200 [00:00<?, ? examples/s]

In [5]:
# Explore the dataset
# For example, print the column names and display a few sample rows
# TODO: your code goes here
print(dataset.column_names)
print(len(dataset['train']['SMILES']))
print(len(dataset['train']['label']))
print(dataset['train'][0])

{'train': ['SMILES', 'label']}
4200
4200
{'SMILES': 'Cn1c(CN2CCN(CC2)c3ccc(Cl)cc3)nc4ccccc14', 'label': 3.54}


In [6]:
# define a PyTorch Dataset class for handling SMILES strings and targets

# TODO: your code goes here
class SMILESDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        smiles = self.dataset[idx]['SMILES']

        encoded = self.tokenizer(smiles, return_tensors='pt', padding='max_length', max_length=512, truncation=True)
        encoded = {k: v.squeeze() for k, v in encoded.items()}

        return {**encoded, 'labels': torch.tensor(self.dataset[idx]['label'], dtype=torch.float32)}

## --- Step 2: Split Dataset ---

As there is only one split (train split) in the original dataset, we need to split the data into training and testing sets by ourselves.

In [7]:
# tokenize the data
# load a pre-trained tokenizer from HuggingFace
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, deterministic_eval=True, trust_remote_code=True)


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenization_molformer_fast.py:   0%|          | 0.00/6.50k [00:00<?, ?B/s]

tokenization_molformer.py:   0%|          | 0.00/9.48k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- tokenization_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- tokenization_molformer_fast.py
- tokenization_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


vocab.json:   0%|          | 0.00/41.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/54.0k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [9]:
encode = tokenizer(dataset['train'][0]['SMILES'], padding='max_length', truncation=True, max_length=512, return_tensors="pt")
encode['attention_mask'].shape

len(dataset['train'])

4200

In [10]:
# split the data into training and test datasets
# TODO: your code goes here
train_indices, test_indices = train_test_split(range(len(dataset['train'])), test_size=0.2, random_state=42)
train_set = Subset(dataset['train'], train_indices)
test_set = Subset(dataset['train'], test_indices)

train_dataset = SMILESDataset(train_set, tokenizer)
test_dataset = SMILESDataset(test_set, tokenizer)

print(f"Train DataLoader with {len(train_dataset)} data points created.")
print(f"Test DataLoader with {len(test_dataset)} data points created.")

Train DataLoader with 3360 data points created.
Test DataLoader with 840 data points created.


In [11]:
# construct Pytorch data loaders for both train and test datasets
BATCH_SIZE = 16 # adjust based on memory constraints

# TODO: your code goes here
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

## --- Step 3: Load Model ---

In [12]:
# load pre-trained model from HuggingFace
model = AutoModel.from_pretrained(MODEL_NAME, trust_remote_code=True)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

configuration_molformer.py:   0%|          | 0.00/7.60k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- configuration_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_molformer.py:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/ibm/MoLFormer-XL-both-10pct:
- modeling_molformer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/187M [00:00<?, ?B/s]

In [13]:
# We need to add a regression head on the language model as we are doing a regression task.

# specify model with a regression head

class MoLFormerWithRegressionHead(nn.Module):
    # TODO: your code goes here
    def __init__(self, model):
        super(MoLFormerWithRegressionHead, self).__init__()
        self.model = model
        self.hidden_size = model.config.hidden_size
        self.regression_head = nn.Linear(self.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids, attention_mask)
        # cls_token = outputs.last_hidden_state[:, 0, :]
        sequence_output = outputs[0]
        cls_token = sequence_output[:, 0, :]
        outputs_head = self.regression_head(cls_token)
        return outputs_head
# instantiate the model
# model = MoLFormerWithRegressionHead(model)

In [14]:
# initialize the regression model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
regression_model = MoLFormerWithRegressionHead(model).to(device)

## --- Step 4: Training ---

In [ ]:
# TODO: your code goes here
num_epochs = 20
optimizer = torch.optim.AdamW(regression_model.parameters(), lr=5e-5)
criterion = nn.MSELoss()
regression_model.train()

for epoch in range(num_epochs):
    total_loss = 0

    for i, data in enumerate(tqdm(train_loader)):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        label = data['labels'].to(device)
        optimizer.zero_grad()
        outputs = regression_model(input_ids, attention_mask)
        loss = criterion(outputs.squeeze(), label)

        loss.backward()
        optimizer.step()
        total_loss += loss.item() * label.shape[0]

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataset)}")

  0%|          | 0/210 [00:00<?, ?it/s]

Epoch 1, Loss: 0.9631072766724087


  0%|          | 0/210 [00:00<?, ?it/s]

Epoch 2, Loss: 0.4960657344687553


  0%|          | 0/210 [00:00<?, ?it/s]

...

Epoch 18, Loss: 0.08823751479919467


  0%|          | 0/210 [00:00<?, ?it/s]

Epoch 19, Loss: 0.08117713609799034


  0%|          | 0/210 [00:00<?, ?it/s]

Epoch 20, Loss: 0.07791138789838269


## --- Step 5: Evaluation ---

In [ ]:
# TODO: your code goes here
regression_model.eval()
total_loss = 0

for i, data in enumerate(tqdm(test_loader)):
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    label = data['labels'].to(device)
    outputs = regression_model(input_ids, attention_mask)
    loss = criterion(outputs.squeeze(), label)
    total_loss += loss.item() * label.shape[0]

print(f"Test Loss: {total_loss / len(test_dataset)}")

  0%|          | 0/53 [00:00<?, ?it/s]

Test Loss: 0.47730408452806017


# 2.Add Unsupervised Finetuning
In this step, you will perform unsupervised fine-tuning on the training dataset. This means the model will leverage only the SMILES strings without any corresponding labels to adapt its understanding of the data distribution. By familiarizing the model with the patterns and structure of the SMILES strings, you can potentially enhance its performance on downstream supervised tasks.

For this fine-tuning, you will use the Masked Language Modeling (MLM) objective, where the model learns to predict randomly masked tokens within the input sequence. Remember to save the fine-tuned model for later use.


In [ ]:
# TODO: your code goes here
from transformers import get_scheduler

# unlabel_dataset = SMILESDataset(train_set, tokenizer, False)
# print(f"Train DataLoader with {len(unlabel_dataset)} unsupervised data points created.")

unsup_model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME,
                                                   deterministic_eval=True,
                                                   trust_remote_code=True).to(device)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                                mlm=True,
                                                mlm_probability=0.15)

train_dataloader = DataLoader(train_dataset,
                              batch_size=16,
                              shuffle=True,
                              collate_fn=data_collator)

optimizer = torch.optim.AdamW(unsup_model.parameters(), lr=5e-5)
num_epochs = 100
num_training_steps = num_epochs * len(train_dataloader)
scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps
)

unsup_model.train()
best_loss = float('inf')
count = 0

for epoch in range(num_epochs):
    total_loss = 0

    for data in tqdm(train_dataloader):
        data = {k: v.to(device) for k, v in data.items()}
        outputs = unsup_model(**data)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        total_loss += loss.item() * data['input_ids'].shape[0]

    epoch_loss = total_loss / len(train_dataset)

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        count = 0
        #save model
        unsup_model.save_pretrained("./finetuned-mlm-model")
        tokenizer.save_pretrained("./finetuned-mlm-token")
        print("Model saved ...")
    else:
        count += 1

    print(f"Epoch {epoch+1}, Loss: {epoch_loss}, Count: {count}")

    if count == 10: # early stop
        print("Early stop !")
        break


  0%|          | 0/210 [00:00<?, ?it/s]

Model saved ...
Epoch 1, Loss: 0.7052506881100791, Count: 0


  0%|          | 0/210 [00:00<?, ?it/s]

Model saved ...
Epoch 2, Loss: 0.3895466801666078, Count: 0


  0%|          | 0/210 [00:00<?, ?it/s]

...

Epoch 98, Loss: 0.03826844466744833, Count: 8


  0%|          | 0/210 [00:00<?, ?it/s]

Epoch 99, Loss: 0.037880165968090296, Count: 9


  0%|          | 0/210 [00:00<?, ?it/s]

Epoch 100, Loss: 0.040032584148658706, Count: 10
Early stop !


# 3.Fine-Tune for Comparison
After performing unsupervised fine-tuning on the training data, we now fine-tune the model on the regression task with the regression head. By comparing the performance of the model before and after unsupervised fine-tuning, you can evaluate how the unsupervised fine-tuning impacts the model's performance on our target task.


In [ ]:
# TODO: your code goes here
from transformers import AutoConfig

# Clear the cache
# config = AutoConfig.from_pretrained("/content/drive/MyDrive/NNTI_project/notebooks/finetuned-mlm-model", trust_remote_code=True)

finetuned_mlm_model = AutoModel.from_pretrained(
    "./finetuned-mlm-model",
    deterministic_eval=True,
    trust_remote_code=True,
)

finetune_model = MoLFormerWithRegressionHead(finetuned_mlm_model).to(device)

num_epochs = 20
optimizer = torch.optim.AdamW(finetune_model.parameters(), lr=5e-5)
criterion = nn.MSELoss()
best_loss = 0.0
count = 0

for epoch in range(num_epochs):
    finetune_model.train()
    train_loss = 0

    for i, data in enumerate(tqdm(train_loader)):
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        label = data['labels'].to(device)
        outputs = finetune_model(input_ids, attention_mask)
        loss = criterion(outputs.squeeze(), label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * label.shape[0]

    epoch_loss = train_loss / len(train_dataset)
    print(f"Epoch {epoch+1}, Loss: {train_loss / len(train_dataset)}")

    if epoch_loss < best_loss:
        best_loss = epoch_loss
        count = 0
    else:
        count += 1

    finetune_model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, data in enumerate(tqdm(test_loader)):
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            label = data['labels'].to(device)
            outputs = finetune_model(input_ids, attention_mask)
            loss = criterion(outputs.squeeze(), label)
            test_loss += loss.item() * label.shape[0]

        print(f"Test Loss: {test_loss / len(test_dataset)}")

    if count == 5: # early stop
        print("Early stop !")
        break

  0%|          | 0/210 [00:00<?, ?it/s]

Epoch 1, Loss: 0.9284282751736187


  0%|          | 0/53 [00:00<?, ?it/s]

Test Loss: 0.8023252027375357


  0%|          | 0/210 [00:00<?, ?it/s]

...

Test Loss: 0.3843427618344625


  0%|          | 0/210 [00:00<?, ?it/s]

Epoch 20, Loss: 0.08563885562831447


  0%|          | 0/53 [00:00<?, ?it/s]

Test Loss: 0.3763835790611449
